# Caso Bae codigo
# Resolución de Restricciones y Requisitos del Problema

## 1. Restricciones de Capacidad y Demanda

**Problema**: Cada vehículo tiene capacidad limitada y debe satisfacer toda la demanda de los clientes.

**Solución implementada**:
- Modelado de entregas parciales permitiendo múltiples visitas a un mismo cliente
- Restricción que garantiza la suma de todas las entregas a un cliente iguale su demanda total
- Control de capacidad por viaje (no acumulativo)

## 2. Autonomía y Gestión de Combustible

**Problema**: Los vehículos tienen autonomía limitada y deben repostar.

**Solución implementada**:
- Variables de combustible que se actualizan en cada tramo del viaje
- Reinicio automático del combustible al volver a un depósito
- Restricciones que garantizan suficiente combustible para cada tramo
- Posibilidad implícita de repostar en depósitos

## 3. Múltiples Viajes por Vehículo

**Problema**: Un mismo vehículo puede necesitar realizar varios viajes para satisfacer toda la demanda.

**Solución implementada**:
- Eliminación de restricciones que limitaban a un solo viaje por vehículo
- Modelado de rutas independientes que comienzan/terminan en depósitos
- Reconstrucción de múltiples rutas por vehículo en post-procesamiento

## 4. Optimización de Costos Operativos

**Problema**: Minimizar los costos totales de operación.

**Solución implementada**:
- Función objetivo que minimiza la distancia total recorrida
- Costo proporcional a la distancia en el cálculo final
- Consideración implícita de costos de combustible a través de la optimización de rutas

## 5. Restricciones Geográficas

**Problema**: Diferentes ubicaciones con distancias variables.

**Solución implementada**:
- Matriz de distancias precomputada usando fórmula Haversine
- Consideración de coordenadas geográficas reales
- Cálculo preciso de distancias entre todos los puntos

## 6. Flexibilidad en la Solución

**Problema**: Balance entre optimalidad y tiempo de cómputo.

**Solución implementada**:
- Configuración de gap de optimalidad del 10%
- Límite de tiempo configurable (12 horas por defecto)
- Capacidad de obtener soluciones factibles aunque no óptimas

## 7. Validación y Verificación

**Problema**: Garantizar que la solución cumpla todos los requisitos.

**Solución implementada**:
- Generación de reporte CSV detallado con todas las métricas
- Visualización geográfica para verificación manual
- Cálculos de validación en post-procesamiento

## 8. Consideraciones Operativas Prácticas

**Problema**: Factibilidad operativa de las rutas generadas.

**Solución implementada**:
- Tiempos de viaje estimados basados en velocidad promedio
- Secuenciación lógica de visitas a clientes
- Consideración de capacidad y autonomía en cada viaje individual


In [ ]:
import pandas as pd
import numpy as np
from pyomo.environ import *
import folium
from math import radians, cos, sin, asin, sqrt
import time
import random
from IPython.display import display as ipy_display

# ------------------------------
# 1. Carga y Procesamiento de Datos
# ------------------------------
print("Cargando datos...")
start_time = time.time()

# Carga optimizada de datos
clients_df = pd.read_csv('clients (1).csv', sep=',', 
                        dtype={'LocationID': 'int32', 'Demand': 'float32', 
                              'Longitude': 'float32', 'Latitude': 'float32'})
depots_df = pd.read_csv('depots (1).csv', sep=',', 
                       dtype={'LocationID': 'int32', 
                             'Longitude': 'float32', 'Latitude': 'float32'})
vehicles_df = pd.read_csv('vehicles (1).csv', sep=',', 
                         dtype={'VehicleID': 'int32', 'Capacity': 'float32', 
                               'Range': 'float32'})

print("Procesando datos...")
demands = clients_df.set_index('LocationID')['Demand'].to_dict()
coords = pd.concat([
    depots_df[['LocationID', 'Longitude', 'Latitude']],
    clients_df[['LocationID', 'Longitude', 'Latitude']]
]).set_index('LocationID').apply(tuple, axis=1).to_dict()

nodes = list(coords.keys())
depots = depots_df['LocationID'].unique().tolist()
clients = clients_df['LocationID'].unique().tolist()

# Función Haversine optimizada
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return 6371 * 2 * asin(sqrt(a))

# Matriz de distancias
distances = {(i, j): haversine(coords[i][0], coords[i][1], coords[j][0], coords[j][1]) 
             for i in nodes for j in nodes if i != j}

# ------------------------------
# 2. Heurística Constructiva para Solución Inicial
# ------------------------------
def greedy_initial_solution():
    solution = {v: [] for v in vehicles_df['VehicleID']}
    unvisited = set(clients)
    vehicle_info = vehicles_df.set_index('VehicleID').to_dict('index')
    
    while unvisited:
        for v_id, info in vehicle_info.items():
            if not unvisited:
                break
                
            # Seleccionar depósito aleatorio
            depot = random.choice(depots)
            current_node = depot
            route = [current_node]
            remaining_capacity = info['Capacity']
            remaining_range = info['Range']
            
            while unvisited and remaining_capacity > 0:
                # Encontrar el cliente más cercano factible
                feasible = []
                for node in unvisited:
                    if demands[node] <= remaining_capacity:
                        dist = distances[(current_node, node)]
                        if dist <= remaining_range:
                            feasible.append((node, dist))
                
                if not feasible:
                    break
                
                # Seleccionar el más cercano
                next_node, dist = min(feasible, key=lambda x: x[1])
                
                route.append(next_node)
                unvisited.remove(next_node)
                remaining_capacity -= demands[next_node]
                remaining_range -= dist
                current_node = next_node
            
            if len(route) > 1:
                solution[v_id].append(route)
    
    return solution

print("Generando solución inicial con heurística greedy...")
initial_solution = greedy_initial_solution()

# ------------------------------
# 3. Construcción del Modelo Pyomo
# ------------------------------
print("Construyendo modelo Pyomo...")
model = ConcreteModel()

# Sets
model.V = Set(initialize=vehicles_df['VehicleID'].unique().tolist(), ordered=True)
model.N = Set(initialize=nodes, ordered=True)
model.P = Set(initialize=depots, ordered=True)
model.D = Set(initialize=clients, ordered=True)

# Parámetros
model.d = Param(model.N, model.N, initialize=distances, default=1000)
model.Q = Param(model.V, initialize=vehicles_df.set_index('VehicleID')['Capacity'].to_dict())
model.q = Param(model.D, initialize=demands)
model.a = Param(model.V, initialize=vehicles_df.set_index('VehicleID')['Range'].to_dict())
model.e = Param(model.V, initialize={v: 1.0 for v in vehicles_df['VehicleID']})

# Variables
model.x = Var(model.V, model.N, model.N, domain=Binary, initialize=0)
model.delivered = Var(model.V, model.D, domain=NonNegativeReals, 
                     bounds=(0, max(vehicles_df['Capacity'])))
model.fuel = Var(model.V, model.N, domain=NonNegativeReals, 
                bounds=(0, max(vehicles_df['Range'])))

# Inicializar variables con solución greedy
print("Inicializando variables con solución heurística...")
for v_id, routes in initial_solution.items():
    for route in routes:
        for i in range(len(route)-1):
            model.x[v_id, route[i], route[i+1]].value = 1
        
        for node in route[1:]:  # Excluir depósito
            if node in model.D:
                model.delivered[v_id, node].value = demands[node]
        
        # Inicializar combustible
        current_fuel = vehicles_df[vehicles_df['VehicleID'] == v_id]['Range'].values[0]
        model.fuel[v_id, route[0]].value = current_fuel
        for i in range(len(route)-1):
            current_fuel -= distances[(route[i], route[i+1])]
            model.fuel[v_id, route[i+1]].value = current_fuel

# Función Objetivo
def obj_rule(m):
    return sum(m.d[i, j] * m.x[v, i, j] for v in m.V for i in m.N for j in m.N if i != j)
model.obj = Objective(rule=obj_rule, sense=minimize)

# Restricciones
def capacity_rule(m, v):
    return sum(m.delivered[v, j] for j in m.D) <= m.Q[v]
model.capacity = Constraint(model.V, rule=capacity_rule)

def demand_rule(m, j):
    return sum(m.delivered[v, j] for v in m.V) == m.q[j]
model.demand = Constraint(model.D, rule=demand_rule)

def link_delivery_rule(m, v, j):
    return m.delivered[v, j] <= m.Q[v] * sum(m.x[v, i, j] for i in m.N if i != j)
model.link_delivery = Constraint(model.V, model.D, rule=link_delivery_rule)

def flow_rule(m, v, j):
    if j in m.P:
        return Constraint.Skip
    return sum(m.x[v, i, j] for i in m.N if i != j) == sum(m.x[v, j, k] for k in m.N if j != k)
model.flow = Constraint(model.V, model.N, rule=flow_rule)

def depart_rule(m, v):
    return sum(m.x[v, p, j] for p in m.P for j in m.N if p != j) <= 1
model.depart = Constraint(model.V, rule=depart_rule)

def init_fuel_rule(m, v):
    return m.fuel[v, list(m.P)[0]] == m.a[v]
model.init_fuel = Constraint(model.V, rule=init_fuel_rule)

def fuel_update_rule(m, v, i, j):
    if i == j:
        return Constraint.Skip
    return m.fuel[v, j] <= m.fuel[v, i] - m.d[i, j] * m.e[v] + 10000 * (1 - m.x[v, i, j])
model.fuel_update = Constraint(model.V, model.N, model.N, rule=fuel_update_rule)

def fuel_suff_rule(m, v, i, j):
    if i == j:
        return Constraint.Skip
    return m.fuel[v, i] >= m.d[i, j] * m.e[v] - 10000 * (1 - m.x[v, i, j])
model.fuel_suff = Constraint(model.V, model.N, model.N, rule=fuel_suff_rule)

# ------------------------------
# 4. Resolución del Modelo
# ------------------------------
print("Resolviendo modelo...")
solver = SolverFactory('highs')
solver.options['mip_rel_gap'] = 0.1
solver.options['time_limit'] = 43200
results = solver.solve(model, tee=True)

# ------------------------------
# 5. Procesamiento de Resultados y Generación de CSV
# ------------------------------
print("\nProcesando resultados y generando archivo CSV...")

def generate_solution_dataframe(model, coords, distances):
    data = []
    
    for v in model.V:
        # Obtener todos los arcos activos para este vehículo
        active_arcs = [(i, j) for i in model.N for j in model.N 
                      if i != j and model.x[v, i, j].value > 0.5]
        
        if not active_arcs:
            continue
            
        # Reconstruir la ruta completa
        route = []
        used_arcs = set()
        
        # Encontrar el punto de partida (depósito)
        start_point = None
        for (i, j) in active_arcs:
            if i in model.P:
                start_point = i
                break
        if start_point is None:
            start_point = active_arcs[0][0]
            
        current_node = start_point
        route.append(current_node)
        
        while True:
            next_node = None
            for (i, j) in active_arcs:
                if i == current_node and (i, j) not in used_arcs:
                    next_node = j
                    used_arcs.add((i, j))
                    break
                    
            if next_node is None:
                break
                
            route.append(next_node)
            current_node = next_node
        
        # Calcular métricas
        total_distance = sum(distances[(route[i], route[i+1])] for i in range(len(route)-1))
        municipalities_visited = [node for node in route if node in model.D]
        demand_satisfied = sum(model.delivered[v, j].value for j in municipalities_visited)
        num_municipalities = len(municipalities_visited)
        
        # Calcular carga inicial (suma de demandas + lo que queda al final)
        initial_load = sum(model.delivered[v, j].value for j in municipalities_visited)
        
        # Obtener capacidad del vehículo
        vehicle_capacity = model.Q[v]
        vehicle_range = model.a[v]
        
        # Calcular tiempo estimado (asumiendo velocidad promedio de 50 km/h)
        estimated_time = total_distance / 50  # en horas
        
        # Calcular costo total (asumiendo $1000 por km como costo operativo)
        total_cost = total_distance * 1000
        
        # Agregar a los datos
        data.append({
            'VehicleId': v,
            'LoadCap': vehicle_capacity,
            'FuelCap': vehicle_range,
            'RouteSequence': ' - '.join(str(node) for node in route),
            'Municipalities': num_municipalities,
            'DemandSatisfied': demand_satisfied,
            'InitialLoad': initial_load,
            'InitialFuel': vehicle_range,
            'Distance': round(total_distance, 2),
            'Time': round(estimated_time, 2),
            'TotalCost': round(total_cost, 2)
        })
    
    return pd.DataFrame(data)

# Generar el DataFrame con los resultados
if results.solver.termination_condition in [TerminationCondition.optimal, TerminationCondition.feasible]:
    solution_df = generate_solution_dataframe(model, coords, distances)
else:
    print("No se encontró solución óptima. Generando CSV con solución heurística...")
    
    # Función para procesar la solución heurística
    def process_heuristic_solution(solution, coords, distances):
        data = []
        
        for v_id, routes in solution.items():
            for route in routes:
                if len(route) < 2:  # Ruta vacía o solo depósito
                    continue
                    
                # Calcular métricas
                total_distance = sum(distances[(route[i], route[i+1])] for i in range(len(route)-1))
                municipalities_visited = [node for node in route if node in clients]
                demand_satisfied = sum(demands[node] for node in municipalities_visited)
                num_municipalities = len(municipalities_visited)
                
                # Obtener información del vehículo
                vehicle_info = vehicles_df[vehicles_df['VehicleID'] == v_id].iloc[0]
                vehicle_capacity = vehicle_info['Capacity']
                vehicle_range = vehicle_info['Range']
                
                # Calcular tiempo y costo
                estimated_time = total_distance / 50  # horas
                total_cost = total_distance * 1000  # $1000 por km
                
                data.append({
                    'VehicleId': v_id,
                    'LoadCap': vehicle_capacity,
                    'FuelCap': vehicle_range,
                    'RouteSequence': ' - '.join(str(node) for node in route),
                    'Municipalities': num_municipalities,
                    'DemandSatisfied': demand_satisfied,
                    'InitialLoad': demand_satisfied,
                    'InitialFuel': vehicle_range,
                    'Distance': round(total_distance, 2),
                    'Time': round(estimated_time, 2),
                    'TotalCost': round(total_cost, 2)
                })
        
        return pd.DataFrame(data)
    
    solution_df = process_heuristic_solution(initial_solution, coords, distances)

# Guardar el DataFrame en un archivo CSV
output_filename = 'verificacion_caso1.csv'
solution_df.to_csv(output_filename, index=False)
print(f"\nArchivo '{output_filename}' generado exitosamente con {len(solution_df)} rutas.")

# Mostrar las primeras filas del DataFrame generado
print("\nVista previa del archivo generado:")
print(solution_df.head())

# ------------------------------
# 6. Visualización de Resultados
# ------------------------------
print("\nGenerando visualización de rutas...")

def visualize_solution(solution, coords, title):
    center = [np.mean([coord[1] for coord in coords.values()]), 
              np.mean([coord[0] for coord in coords.values()])]
    m = folium.Map(location=center, zoom_start=12)
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
              '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    
    total_distance = 0
    for idx, route_data in enumerate(solution):
        color = colors[idx % len(colors)]
        
        # Parsear la secuencia de la ruta
        route = [int(node.strip()) for node in route_data['RouteSequence'].split(' - ')]
        
        # Calcular distancia de la ruta
        route_dist = route_data['Distance']
        total_distance += route_dist
        
        # Dibujar ruta
        folium.PolyLine(
            [coords[node] for node in route],
            color=color, weight=3, opacity=0.8,
            popup=f'Vehículo {route_data["VehicleId"]} - Distancia: {route_dist:.2f}km'
        ).add_to(m)
        
        # Marcadores
        for node in route:
            folium.CircleMarker(
                location=(coords[node][1], coords[node][0]),
                radius=5, color=color, fill=True,
                popup=f'Nodo {node}'
            ).add_to(m)
    
    print(f"{title} - Distancia total: {total_distance:.2f} km")
    return m

# Visualizar solución
if not solution_df.empty:
    m = visualize_solution(solution_df.to_dict('records'), coords, "Solución Final")
    display(m)
else:
    print("No hay rutas para visualizar.")

print(f"\nTiempo total de ejecución: {time.time() - start_time:.2f} segundos")